# CATEGORIES SCRIPT FOR DATA STUDIO MARKETING DASHBOARD

### 1. Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from pymongo import MongoClient
import pandas_gbq as pgbq

### 2. Creating date parameters for dynamic data filtering from DB

In [2]:
threeMonths = str(date.today() - timedelta(days= np.floor(365/2)/2))

### 3. Connecting to DB and extracting order and category data 

- Connection to Mongo client and extraction

In [3]:
# Connecting to mongodb client
client = MongoClient("mongodb://12.3.45.678:27017/?readPreference=primary&directConnection=true&ssl=false")
# Choosing collection from DB for aggregation
ordersColl= client.dotComOrders.dailyOrders
# Category aggregation for three months
categories = ordersColl.aggregate([
    {
        '$match': {
            'site': {
                '$in': [
                    'site1', 'site2' # Matching only UAE & Oman data
                ]
            }, 
            'created': {
                '$gt': threeMonths # Matching of data from last 3 months
            }
        }
    }, {
        '$unwind': {
            'path': '$consignments', 
            'preserveNullAndEmptyArrays': True
        }
    }, {
        '$unwind': {
            'path': '$consignments.entries', 
            'preserveNullAndEmptyArrays': True
        }
    }, {
        '$group': {
            '_id': {
                'code': '$code', 
                'productType': '$consignments.entries.orderEntry.product.erpCategory' # Grouping by order code & product type
            }, 
            'date': {
                '$first': {
                    '$toDate': '$created'
                }
            }, 
            'site': {
                '$first': '$site'
            }, 
            'orders': {
                '$sum': 1
            }, 
            'quantity': {
                '$first': '$consignments.entries.orderEntry.quantity'
            }, 
            'value': {
                '$first': '$consignments.entries.orderEntry.totalPrice.value'
            }
        }
    }, {
        '$lookup': {
            'from': 'articles', 
            'localField': '_id.productType', 
            'foreignField': 'mat group', 
            'as': 'articles' # Joining with articles collection for product type description
        }
    }, {
        '$project': {
            '_id': 0, 
            'date': 1, 
            'site': 1, 
            'productType': {
                '$first': '$articles.Product type'
            }, 
            'orders': 1, 
            'quantity': 1, 
            'value': 1
        }
    }
],allowDiskUse = True)
# Printing cursor
print(categories)

- Convertion of cursor to dataframe

In [4]:
categoriesList = list(categories) # Conversion of cursor to list
categoriesDf = pd.DataFrame(categoriesList) # Conversion of list to dataframe

- Casting data types for efficient memory usage

In [5]:
categoriesDf["site"],categoriesDf["orders"],categoriesDf["quantity"],categoriesDf["value"],categoriesDf["productType"] = categoriesDf["site"].astype("category"),categoriesDf["orders"].astype("int8"),categoriesDf["quantity"].astype("float32"),categoriesDf["value"].astype("float32"),categoriesDf["productType"].astype("category")

### 4. Exporting dataframe to bigquery

In [6]:
categoriesDf.to_gbq(destination_table="ordersdata.dailycategory", project_id="orderdash", if_exists="replace")

### 5. Closing client

In [7]:
client.close()